<a href="https://colab.research.google.com/github/yeeecheng/Update-Monthly-Option-Data/blob/main/%E6%9C%88%E9%81%B8%E6%93%87%E6%AC%8A%E8%B3%87%E6%96%99%E5%8D%B3%E6%99%82%E6%9B%B4%E6%96%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 選擇權(月)歷史資料即時更新

In [ ]:
import urllib3
from bs4 import BeautifulSoup

import pandas as pd
from datetime import date
import sys
import datetime
import calendar

import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import files
from google.colab import drive 

import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()


update 2022/07 data
update 2022/08 data
update 2022/09 data


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def read_file(file_name):
  
  df = pd.read_csv(file_name)

  row,col= df.shape

  #資料日期
  data_date = df['最後結算日'][row-1].split('-')
  #目前日期
  cur_date = str(datetime.date.today()).split('-')

  #判斷是否為最新
  if(data_date == cur_date): 
    sys.exit("已是最新版本!")

  return df,data_date,cur_date

In [ ]:
def post_http(data_date):

  #每月週選擇權資料爬蟲


  http = urllib3.PoolManager()
  url = "https://www.taifex.com.tw/cht/5/fSPDataDown"
  res = http.request(
      'POST',
      url,
      fields={
          'dlFileType' : 3,
          'start_year' : data_date[0],
          'start_month' : data_date[1],
          'end_year': data_date[0],
          'end_month' : data_date[1]
      }
  )

  html_doc = res.data
  #html_doc

  soup = BeautifulSoup(html_doc, 'html.parser')
  soup

  soup_str = str(soup)
  lines = soup_str.split('\r\n')

  return lines

In [ ]:
def check_updated_date(data_date,lines):
  
  #找到要從哪裡更新資料
  start_idx = 0
  for i in range(len(lines)-2,0,-1):
  
      day_list = lines[i].split(',')[0].split('/')
      if(int(day_list[2])>int(data_date[2])):
        start_idx = i
        break

  return start_idx



In [ ]:
def process_new_data(new_data,lines,start_idx):
  
  
  for i in range(start_idx,0,-1):

    detail_list = lines[i].split(',')

    #防止出現一些例外   
    if(len(detail_list)!=5):
        continue


    if(detail_list[2]!='TXO'):
        continue

    new_data.append(detail_list)
    
  return new_data

In [ ]:
def concat_data(df,new_data):
  
  new_df = pd.DataFrame(new_data,columns=df.columns)
  df = pd.concat([df,new_df],ignore_index=True)

  

  return df

In [ ]:
def create_csv(df,file_name):
  
  df.to_csv(file_name, encoding = 'utf-8-sig')
  

In [ ]:
def process_Date(df):
  for i in range(len(df)):

    df['最後結算日'][i] = df['最後結算日'][i].replace('/','-')

  return df


In [ ]:
def process_csv(df):
  for i in range(len(df)):

    df['最後結算日'][i] = df['最後結算日'][i].replace('-','')
    df['契約月份'][i] = df['契約月份'][i].replace('W','7')

  return df

In [ ]:
def read_file_OPT(file_name):

  df = pd.read_csv(file_name)
  row,col= df.shape
  doc_date = df['交易日期'][row-1].split('-')
  cur_date = date.today().strftime('%Y-%m-%d').split('-')

  if doc_date == cur_date:
      sys.exit("檔案已經是最新的版本")

  return df,doc_date,cur_date

In [ ]:
def post_http_OPT(year,month,day,lastDay):
  http = urllib3.PoolManager()
  url = "https://www.taifex.com.tw/cht/3/optDataDown"
  res = http.request(
      'POST',
      url,
      fields={
          'down_type': 1,
          'commodity_id': 'TXO',
          'queryStartDate' : f'{year}/{month}/{day}',
          'queryEndDate': f'{year}/{month}/{lastDay}'
      }
  )

  html_doc = res.data
  soup = BeautifulSoup(html_doc, 'html.parser')
  soup_str = str(soup)
  lines = soup_str.split('\r\n')
  
  return lines


In [ ]:
def check_updated_date_OPT(doc_date):

  year = int(doc_date[0])
  month = int(doc_date[1])
  day = int(doc_date[2]) + 1
  if day > calendar.monthrange(year, month)[1]:
    month+=1

  return year,month,day




In [ ]:
def addToFile(file, what):
  
  open(file, 'a',encoding = 'utf-8-sig').write(what)


In [ ]:
def process_csv_OPT(df):

  df['交易日期']=df['交易日期'].str.replace('-','')
  df['到期月份(週別)']=df['到期月份(週別)'].str.replace('W','7')
  
  return df

In [ ]:
def create_csv_OPT(df,file_name):
  
  df.to_csv(file_name, encoding = 'utf-8-sig')

In [ ]:

#月選結算

df , data_date , cur_date = read_file('https://drive.google.com/uc?export=download&id=1-0rjc0jHCZB2w5oAEtB85VMzP0AVt66x')

start_date = data_date.copy()
new_data=[]

#以每個月為單位更新資料
while(start_date[0]!=cur_date[0] or int(start_date[1]) <= int(cur_date[1])):
  
  print(f"update {start_date[0]}/{start_date[1]} data")
  lines = post_http(start_date)
  start_idx=len(lines)-2

  if(start_date == data_date):
    start_idx = check_updated_date(start_date,lines)

  new_data = process_new_data(new_data,lines,start_idx)

  month = int(start_date[1])+1
  year = int(start_date[0])
  if(month >12):
    year+=1
    month=1
  start_date[2]="01"
  start_date[1]=str(month).zfill(2)
  start_date[0]=str(year).zfill(4)



#結合資料
df =concat_data(df,new_data)


#更新原始資料
file_name="data_month.csv"
df['契約月份'] = df['契約月份'].astype(str)
df = df[~df['契約月份'].str.contains("W")]
df['契約月份'] = df['契約月份'].str.strip()
df.reset_index(drop=True, inplace=True)
df = process_Date(df)
df.set_index('最後結算日' , inplace=True)
create_csv(df,file_name)



#下載
file_name="月選擇權結算價.csv"
df.reset_index(inplace=True)
df = process_csv(df)
df.set_index('最後結算日' , inplace=True)
create_csv(df,file_name)
files.download(file_name)


In [ ]:
#月選擇權歷史
df ,doc_date,cur_date =read_file_OPT("https://drive.google.com/uc?export=download&id=1Oza020DlaM7kQp9gtLCC0A3KG1Iym_vi")
#找到要更新時間
year,month,day =check_updated_date_OPT(doc_date)

array_csv = ''
#以一個月為單位更新資料
while(year != int(cur_date[0]) or month <= int(cur_date[1])):

    if(month == 13):
      year += 1
      month = 1
    
    if(year != int(doc_date[0]) or month != int(doc_date[1])):
      day=1
    
    lastDay = cur_date[2] if year == cur_date[0] and month == cur_date[1] else calendar.monthrange(year, month)[1]

    lines = post_http_OPT(year,month,day,lastDay)

    for i in range(1, len(lines)-1):
        line = lines[i].split(',')
        if line[17] == '一般': 
          line[0] = line[0].replace('/','-')
          for i in range(0,19):
            array_csv+=line[i]+','
          array_csv+=line[19]+'\n'
        
    month+=1

file_name ="data_month_OPT.csv"
df.set_index('交易日期' , inplace=True)
create_csv_OPT(df,file_name)


#更新原始資料
addToFile('data_month_OPT.csv', array_csv)


#查表
TXO_OPT = pd.read_csv('data_month_OPT.csv')
TXO_OPT['到期月份(週別)'] = TXO_OPT['到期月份(週別)'].astype(str) 
TXO_OPT['到期月份(週別)']=TXO_OPT['到期月份(週別)'].str.strip()
TXO_OPT = TXO_OPT.set_index(pd.to_datetime(TXO_OPT["交易日期"])) #將日期置為索引




Date = pd.read_csv('data_month.csv') 
Date['最後結算日']= pd.to_datetime(Date['最後結算日'])    #將X軸判斷為時間日期
Date['契約月份'] = Date['契約月份'].astype(str)




#資料查表拼接

df = pd.DataFrame()   


first = TXO_OPT[:Date['最後結算日'][0]]
first = first[first['到期月份(週別)']== Date['契約月份'][0]]
df = df.append(first, ignore_index = True)



number = len(Date.index)

for i in range(0,number-1):
  option = TXO_OPT[Date['最後結算日'][i]:Date['最後結算日'][i+1]]
  option = option[option['到期月份(週別)']== Date['契約月份'][i+1]]
  df = df.append(option, ignore_index = True)



search = TXO_OPT[TXO_OPT.index== Date['最後結算日'].iloc[-1]]
search = search[~search['到期月份(週別)'].str.contains(Date['契約月份'].iloc[-1])]


add = TXO_OPT[Date['最後結算日'].iloc[-1]:]
add = add[add['到期月份(週別)']== search['到期月份(週別)'][0]]

df = df.append(add, ignore_index = True)


df.set_index('交易日期' , inplace=True)
#生成、下載
file_name ="data_week_OPT(Final).csv"
create_csv_OPT(df,file_name)

df.reset_index(inplace=True)


df = process_csv_OPT(df)
df.set_index('交易日期' , inplace=True)

#生成、下載
file_name ="月選擇權歷史資料.csv"
create_csv_OPT(df,file_name)
files.download(file_name)